Mars ain't the kind of place to raise a kid  
In fact, it's cold as hell.
           - Elton John, "Rocket Man"

In [1]:
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
import pandas as pd
import time

## Get Mars News

In [2]:
executable_path = {"executable_path" : "chromedriver.exe"}
browser = Browser("chrome", **executable_path, headless=False)


In [3]:
url = 'https://mars.nasa.gov/news/'
browser.visit(url)

html = browser.html
soup = bs(html, 'html.parser')



In [4]:
news_title = soup.find("div", class_="content_title").text
print(news_title)
news_p     = soup.find("div", class_="article_teaser_body").text
print(news_p)


For InSight, Dust Cleanings Will Yield New Science
Wind can be crucial to clearing dust from spacecraft solar panels on Mars. With InSight's meteorological sensors, scientists get their first measurements of wind and dust interacting "live" on the Martian surface.  


## Get Mars Featured Image

In [5]:
url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(url)
browser.click_link_by_partial_text("FULL IMAGE")
time.sleep(3)
browser.click_link_by_partial_text("more info")

In [6]:
html = browser.html
soup = bs(html, 'html.parser')
   
featured_image = soup.find("figure", class_="lede")
print(featured_image)

<figure class="lede">
<a href="/spaceimages/images/largesize/PIA18640_hires.jpg"><img alt="The star on this image from the NASA's Terra spacecraft indicates the eipcenter of a magnitude 6.1 earthquake which truck in southern China's Yunnan province, toppling thousands of homes and causing numerous casualties." class="main_image" src="/spaceimages/images/largesize/PIA18640_hires.jpg" title="The star on this image from the NASA's Terra spacecraft indicates the eipcenter of a magnitude 6.1 earthquake which truck in southern China's Yunnan province, toppling thousands of homes and causing numerous casualties."/></a>
</figure>


In [7]:
featured_image_url = "https://www.jpl.nasa.gov" + featured_image.find("a")["href"]
print(featured_image_url)


https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA18640_hires.jpg


## Get Mars Weather

In [8]:
url = 'https://twitter.com/marswxreport?lang=en'
response = requests.get(url)
soup = bs(response.text, 'lxml')

In [9]:
def getText(parent):
    return ''.join(parent.find_all(text=True, recursive=False)).strip()

# a little extra rigamoroll required because we want the text from the parent "p" but no descendents

result = soup.find("p", class_="tweet-text")
weather_report = getText(result)
print(weather_report)

InSight sol 156 (2019-05-05) low -99.2ºC (-146.6ºF) high -18.1ºC (-0.6ºF)
winds from the SW at 4.7 m/s (10.5 mph) gusting to 13.8 m/s (30.8 mph)
pressure at 7.40 hPa


## Get Mars Facts

In [10]:
url = "https://space-facts.com/mars/"
response = requests.get(url)
soup = bs(response.text, "lxml")


In [11]:
result_labels = soup.find_all("td", class_="column-1")
result_values = soup.find_all("td", class_="column-2")

In [12]:
result_labels_text = []
result_values_text = []
for rlabel in result_labels:
    result_labels_text.append(rlabel.text)
for rvalue in result_values:
    result_values_text.append(rvalue.text)

In [13]:
mars_df = pd.DataFrame({"Stats": result_labels_text,
                        "Values":  result_values_text})
print(mars_df)

                   Stats                         Values
0   Equatorial Diameter:                     6,792 km\n
1        Polar Diameter:                     6,752 km\n
2                  Mass:  6.42 x 10^23 kg (10.7% Earth)
3                 Moons:            2 (Phobos & Deimos)
4        Orbit Distance:       227,943,824 km (1.52 AU)
5          Orbit Period:         687 days (1.9 years)\n
6  Surface Temperature:                   -153 to 20 °C
7          First Record:              2nd millennium BC
8           Recorded By:           Egyptian astronomers


In [14]:
mars_df.set_index("Stats",inplace=True)
display(mars_df)

,Values
Stats,
Equatorial Diameter:,"6,792 km\n"
Polar Diameter:,"6,752 km\n"
Mass:,6.42 x 10^23 kg (10.7% Earth)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.52 AU)"
Orbit Period:,687 days (1.9 years)\n
Surface Temperature:,-153 to 20 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [15]:
mars_facts_html = mars_df.to_html()
display(mars_facts_html)

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Values</th>\n    </tr>\n    <tr>\n      <th>Stats</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km\\n</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km\\n</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.42 x 10^23 kg (10.7% Earth)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.52 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)\\n</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-153 to 20 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    <

## Get Hemisphere Images

In [16]:
url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url)

In [17]:
hemisphere_list = []

hemispheres = ["Cerberus", "Schiaparelli", "Syrtis Major", "Valles Marineris"]
for x in range(0,4):
    browser.click_link_by_partial_text(hemispheres[x])
    
    html = browser.html
    soup = bs(html, 'html.parser')
    
    img_url = "https://astrogeology.usgs.gov" + (soup.find("img", class_="wide-image")["src"])
    title = (soup.find("h2", class_="title").text)    
    
    
    hemisphere_dict = {"title": title, "img_url":img_url}
    hemisphere_list.append(hemisphere_dict)
    
       
    browser.back()
    
    

In [18]:
browser.quit()

In [19]:
display(hemisphere_list)

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'}]